In [ ]:
import json

import pandas as pd

from shapely.geometry import Point
from shapely.geometry import Polygon

from CCAgT_utils.converters.CCAgT import CCAgT_Annotations
from CCAgT_utils.converters.CCAgT import CCAgT_Helper

In [ ]:
data = [{'image_name': 'A_xxx', 'geometry': Point(1, 1), 'category_id': 3},
        {'image_name': 'A_yyy', 'geometry': Point(10, 10), 'category_id': 3},
        {'image_name': 'A_yyy', 'geometry': Polygon([(40, 40), (50, 50), (50, 40)]), 'category_id': 1},
        {'image_name': 'B_yyy', 'geometry': Point(10, 10), 'category_id': 3},
        {'image_name': 'B_yyy', 'geometry': Point(2000, 2000), 'category_id': 3}]

df = pd.DataFrame(data)
df

In [ ]:
ccagt_ann = CCAgT_Annotations(df)

In [ ]:
df['slide_id'] = ccagt_ann.get_slide_id()
df

In [ ]:
df['geo_type'] = ccagt_ann.geometries_type()
df

In [ ]:
sat_series = df.loc[(df['category_id']==3) & (df['geo_type']=='Point'), 'geometry']

df.loc[(df['category_id']==3) & (df['geo_type']=='Point'), 'geometry'] = ccagt_ann.satellite_point_to_polygon(sat_series)

df['geo_type'] = ccagt_ann.geometries_type()

df

In [ ]:
df['geometry'] = ccagt_ann.fit_geometries_to_image_boundary()
df

In [ ]:
df.dropna(axis=0, subset=['geometry'], inplace=True)


In [ ]:
df['area'] = ccagt_ann.geometries_area()
df

In [ ]:
df['image_id'] = ccagt_ann.generate_ids(df['image_name'])
df

In [ ]:
ccagt_ann.df

In [ ]:
helper_path = '../../data/samples/categories_helper.json'
with open(helper_path, 'r') as hf:
    categories_helpper = json.load(hf)

categories_helpper[0], len(categories_helpper)


In [ ]:
ccagt_helper = CCAgT_Helper(categories_helpper)

In [ ]:
ccagt_helper.name_by_category_id

In [ ]:
df = ccagt_ann.delete_by_area(ccagt_helper, ignore_categories={3})
df